***
## Добавляем модель тегов в проект

Начнём с создания модели `Tag`: добавьте описание модели в *birthday/models.py*, а в модели `Birthday` опишите новое поле типа `ManyToManyField()`.


In [ ]:
# birthday/models.py
...

class Tag(models.Model):
    tag = models.CharField('Тег', max_length=20)

class Birthday(models.Model):
    ...
    tags = models.ManyToManyField(
        Tag,
        verbose_name='Теги',
        blank=True,
        help_text='Удерживайте Ctrl для выбора нескольких вариантов'
    ) 

Зарегистрируйте модель `Tag` в админке и добавьте в неё несколько тегов, например — «Друг», «Коллега», «Родственник» (в единственном числе, иначе при отображении они будут странно выглядеть).

![alt text](https://pictures.s3.yandex.net/resources/223_1687356795.png)

Вместо содержимого поля `tag` модели `Tag` в поле выбора отображается имя модели и id объекта. В админ-зоне в списке тегов — та же картина.

В полях выбора формы и для отображения объектов в админке используется строковое представление объекта, его возвращает метод `__str__()` модели. По умолчанию `__str__()` возвращает строку вида `<имя_модели object id_объекта>`. 

Чтобы строка была более информативной — надо переопределить в модели метод `__str__()` так, чтобы он возвращал какое-то понятное значение (в нашем случае — содержимое поля `tag`):


In [ ]:

# birthday/models.py
...

class Tag(models.Model):
    tag = models.CharField('Тег', max_length=20)

    # Переопределяем метод:
    def __str__(self):
        return self.tag
... 


Результат должен быть таким:

![alt text](https://pictures.s3.yandex.net/resources/224_1687356814.png)


***
## Теги на странице

Выведем теги в шаблон. Если записи о дне рождения присвоены теги, то после имени и дня рождения должна отобразиться строка вида 

> ```<список_тегов> пользователя <username_автора_записи> ```

Если у записи несколько тегов — они должны быть выведены через запятую; в конце строки должен стоять `username` автора записи.

К одному объекту `birthday` может быть привязано несколько меток, вывести их в шаблон можно разными способами.

Например, можно обратиться в шаблоне к набору тегов, связанных с объектом `birthday` — `birthday.tags.all`, и перебрать теги в цикле:


In [ ]:
...
      <div class="col-10">  
        <div>
          {{ birthday.first_name }} {{ birthday.last_name }} - {{ birthday.birthday }}<br>
          <a href="{% url 'birthday:detail' birthday.id %}">Сколько до дня рождения?</a>
        </div>

        <!-- Начало нового блока кода -->
        <div>
          <!-- Тег spaceless убирает из строки лишние пробелы 
            и переводы строк -->
          {% spaceless %}
            <!-- Цикл по тегам записи -->
            {% for tag in birthday.tags.all %}
              <!-- Если работаем с первым элементом цикла... -->
              {% if forloop.first %}
                <!-- ...выводим название тега с заглавной буквы -->
                <span>{{ tag.tag|title }}</span>
              {% else %}
                <!-- Если элемент не первый — пишем тег с маленькой буквы -->
                <span>{{ tag.tag|lower }}</span>
              {% endif %}
              <!-- Если обрабатываем не последний элемент цикла... -->
              {% if not forloop.last %}
                <!-- ...после него ставим запятую с пробелом -->
                <span>, </span>
              {% else %}
                <!-- После последнего элемента выводим username пользователя -->
                пользователя {{ birthday.author.username }}
              {% endif %}
            {% endfor %}
          {% endspaceless %}
        </div>
        <!-- Конец нового блока кода -->

        {% if birthday.author == user %}
          <div>
            <a href="{% url 'birthday:edit' birthday.id %}">Изменить запись</a> | <a href="{% url 'birthday:delete' birthday.id %}">Удалить запись</a>
          </div>
        {% endif %}
      </div>

      {% if not forloop.last %}
        <hr class="mt-3">
      {% endif %}
    </div> 


![alt text](https://pictures.s3.yandex.net/resources/225_1687356826.png)

[Установите в проект Django Debug Toolbar](https://code.s3.yandex.net/backend-developer/learning-materials/%D0%A3%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%BA%D0%B0_DjDT.html), сейчас он очень пригодится.

Откройте страницу со списком дней рождения и загляните в Django Debug Toolbar во вкладку SQL. 

На скриншотах отображено количество запросов, отправленных для отображения десяти объектов на странице.

Чтобы отобразить десять объектов — Django отправил более двадцати запросов к базе данных!

![alt text](https://pictures.s3.yandex.net/resources/227_1687356861.png)

Расточительное расходование ресурсов БД до добра не доведёт. Если страницу будут одновременно просматривать сто пользователей — к базе полетит более двух тысяч запросов. А это большие расходы ресурсов, сайт может начать тормозить.

***
## Источник лишних запросов

Класс, управляющий страницей, запрашивает в базе данных список объектов `Birthday` и передаёт их в шаблон. На этом этапе всё хорошо: эта операция выполняется в один запрос к базе данных.

Но в шаблоне требуется отобразить не только информацию из модели `Birthday`. Для каждого объекта `Birthday` надо получить:

* данные из связанной модели `User`. Запрос к модели юзера отправляется для вывода `username` в строке с метками и для проверки авторства записи — выводить ли ссылки на редактирование и удаление записи;

* список тегов: значения поля `tag` модели `Tag`.

В объектах, которые переданы в шаблон, этой информации нет: в них есть только ссылки на объект пользователя, а связь между моделями `Birthday` и `Tag` вообще хранится в другой таблице. 

В подобных случаях шаблонизатор Django умеет сам запрашивать недостающую информацию из базы данных.

На каждой итерации цикла `{% for birthday in page_obj %}` шаблонизатор обнаруживает, что ему нужна дополнительная информация, — и инициирует запросы к моделям `Tag` и `User`. 

С одной стороны — хорошо, фреймворк сам получил и вывел недостающую информацию.

Но при этом Django отправляет по два дополнительных запроса для каждого объекта; это расточительно.

***
## Оптимизируем запросы при связи «многие-ко-многим»

Модели `Birthday` и `Tag` объединены связью «многие-ко-многим», эта связь реализуется через промежуточную таблицу. Через эту таблицу каждая запись таблицы с днями рождения может быть связана с несколькими записями из таблицы с тегами. 

Самый простой вариант получения связанных данных Django реализует очень простым, но затратным способом: он пытается для каждой записи `birthday` отдельным запросом получить связанные объекты из таблицы с тегами.

Более экономный по числу запросов вариант будет таким: 

1. **Первым запросом** получить записи из таблицы с днями рождения — именно это и делает по умолчанию CBV `ListView`.

2. Из полученных записей получить их id — и сделать второй запрос, в котором получить те записи из таблицы с тегами, которые связаны с полученными id.

3. Объединить результаты между собой: привязать к записям, полученным в первом запросе, соответствующие записи, полученные во втором запросе.

В итоге для получения связанных объектов потребуется не по одному запросу на каждый полученный объект `birthday`, а лишь один дополнительный запрос на всё, независимо от того, сколько на странице объектов `birthday`.

***
## Предзагрузка связанных объектов: prefetch_related

По умолчанию класс `BirthdayListView` запрашивает все объекты модели, указанной в атрибуте `model`. В теле класса запрос не описывается в явном виде — он генерируется под капотом: там вызывается метод `Birthday.objects.all()`. Этот метод возвращает QuerySet с полным списком объектов модели; объекты в QuerySet будут отсортированы в соответствии со свойством `ordering = 'id'`.

Но запрос, который отправляет `BirthdayListView` к базе данных, можно описать в явном виде и перенастроить его; это делается в атрибуте `queryset`. 

Опишем запрос для получения всех объектов модели `Birthday` и применим метод `prefetch_related()` — через него запросим связанные объекты из модели `Tag`. 

При вызове `prefetch_related()` в него передаётся имя поля, через которое модель `Birthday` связана с `Tag`:


In [ ]:
class BirthdayListView(ListView):
    model = Birthday
    # По умолчанию этот класс 
    # выполняет запрос queryset = Birthday.objects.all(),
    # но мы его переопределим:
    queryset = Birthday.objects.prefetch_related('tags')
    ordering = 'id'
    paginate_by = 10 

***
## При таком вызове Django ORM отправит два SQL-запроса.

1. Сперва будет отправлен запрос для получения записей из таблицы `birthday_birthday` — точно такой же, какой отправлялся и до того, как мы добавили в приложение систему тегов.

    Этот запрос вернёт из базы первые десять записей о днях рождения. Их `id` могут быть разными — в зависимости от того, как вы заполняли базу и сколько объектов удалили. Для простоты предположим, что из таблицы `birthday_birthday` получены записи с id от 1 до 10.

2. После этого будет отправлен дополнительный запрос к таблице с тегами `birthday_tag`; в инструкцию WHERE будет передан список `id` тех объектов дней рождения, которые были получены при первом запросе. 

    По этим `id` будут получены записи таблицы `birthday_tag`, связанные с полученными объектами `Birthday` через промежуточную таблицу `birthday_birthday_tags`.

![alt text](https://pictures.s3.yandex.net/resources/228_1687356879.png)


3. Когда результаты второго запроса будут получены, Django объединит результаты обоих запросов. Будет создан QuerySet со списком объектов `Birthday`: у каждого объекта `Birthday` в поле `tags` будет храниться QuerySet со списком объектов `Tag`, связанных с этим днём рождения. Теперь можно обращаться к связанным объектам, не отправляя дополнительных запросов к БД.

***
## Оптимизируем запросы при связи «многие-к-одному»

Модели `Birthday` и `User` связаны «один-ко-многим», и это позволяет справиться с лишними запросами с помощью метода `select_related()`. Этот метод объединит запросы к обеим таблицам с помощью инструкции `JOIN` — и данные об авторах будут загружены одновременно с записями о днях рождения.

Допишем запрос в атрибуте `queryset` класса `BirthdayListView`: прямо в него добавим запрос к объектам, связанным с `Birthday` через поле author.


In [ ]:
class BirthdayListView(ListView):
    model = Birthday
    queryset = Birthday.objects.prefetch_related(
        'tags'
    ).select_related('author')
    ordering = 'id'
    paginate_by = 10 

Теперь на странице */birthday/list/* будет видно, что количество запросов уменьшилось сразу на десять штук (по числу выведенных на страницу записей о днях рождения). 

Из класса `BirthdayListView` всё так же отправляются два запроса, но в первый из них добавлен оператор `JOIN`: к записям из таблицы `birthday_birthday` будут присоединены связанные записи из таблицы `auth_user`.

***
## Оптимизировали, оптимизировали — и что?

Django Debug Toolbar показывает, что мы избавились от лишних запросов; приложение работает быстрее, нагрузка на базу в разы меньше!

![alt text](https://pictures.s3.yandex.net/resources/232_1687356943.png)

**Для получения данных из моделей, связанных «один-ко-многим»**, можно применять метод `select_related()`, он позволит снизить количество выполняемых запросов к базе. «Под капотом» при этом выполняется SQL-запрос с использованием JOIN.

**Для получения данных из моделей, связанных «многие-ко-многим»**, можно применять метод `prefetch_related()`, он позволит снизить количество выполняемых запросов к базе. При этом под капотом последовательно выполняются два SQL-запроса, а потом их результат объединяется в один QuerySet.

Такая оптимизация не всегда применима. Например, если на страницу выводится не список объектов, а только один объект, то никакой экономии в запросах `prefetch_related()` не даст. 

Приведённый пример — лишь один из вариантов использования метода `prefetch_related()`, но применять его можно по-разному. Более подробно о методе можно почитать [в документации](https://docs.djangoproject.com/en/5.1/ref/models/querysets/#prefetch-related).

***
## Бонус-лайфхак для любопытных: делаем код шаблона короче

Собрать несколько тегов в единую строку можно с помощью фильтра шаблонизатора `join`. Он работает так же, как метод `.join()` в Python, — объединяет в единую строку элементы списка или иной последовательности.

В шаблоне после названия фильтра через двоеточие в кавычках указывается набор символов, который будет добавлен между элементами списка в результирующей строке.



<!-- names = ['Гена', 'Чебурашка', 'Шапокляк', 'Лариска'] -->

{{ names|join:", " }}
<!-- Выведет: Гена, Чебурашка, Шапокляк, Лариска -->

{{ names|join:" (◕_◕) " }}
<!-- Выведет: Гена (◕_◕) Чебурашка (◕_◕) Шапокляк (◕_◕) Лариска --> 


Метод модели `birthday.tags.all` возвращает список объектов модели `Tag`. Если к полученному списку объектов применить строковый метод `join`, то объекты будут переданы в этот метод в строковом представлении.

Строковое представление объекта модели `Tag` — это название тега, содержимое поля `tag` (ведь в модели переопределён метод `__str__()`).

Итак, если список объектов модели `Tag` передать в метод `join` — вернётся строка с названиями тегов.

Вместо цикла, перебирающего объекты в `birthday.tags.all`, можно обработать этот список фильтром join. Код станет значительно короче: 


In [ ]:

<!-- templates/birthday/birthday_list.html -->
...
    <div class="col-10">  
      <div>
        {{ birthday.first_name }} {{ birthday.last_name }} - {{ birthday.birthday }}<br>
        <a href="{% url 'birthday:detail' birthday.id %}">Сколько до дня рождения?</a>
      </div>

      <!-- Начало нового блока кода -->
      <div>
        <!-- Чтобы сократить количество кода —
          введём переменную all_tags, в которой будут лежать все теги объекта -->
        {% with all_tags=birthday.tags.all %}
          <!-- Если у записи есть хоть один тег -->
          {% if all_tags %}
            <!-- Выводим теги через запятую, самую первую букву делаем заглавной -->
            {{ all_tags|join:", "|lower|capfirst }} 
            <!-- Также выводим username пользователя -->
            пользователя {{ birthday.author.username }}
          {% endif %}
        {% endwith %}
      </div>
      <!-- Конец нового блока кода -->

      {% if birthday.author == user %}
        <div>
          <a href="{% url 'birthday:edit' birthday.id %}">Изменить запись</a> 
          | <a href="{% url 'birthday:delete' birthday.id %}">Удалить запись</a>
        </div>
      {% endif %}
    </div>
... 


Последовательность фильтров `{{ all_tags|join:", "|lower|capfirst }}` 

* объединяет все теги через запятую,

* делает все буквы внутри полученной строки маленькими,

* делает заглавной первую букву строки.